# Assignment 4

```
--Var olan minimum indirim değeri kontrol parametresi olarak kabul edilmiştir 
--[0.05]  total_quantity
--Var olan minimum indirim sonrasındaki indirimlerin quantity ortalaması alınmıştır 
--[0.07],[0.10],[0.20]  Avg_Last_3_Discount
--Quantity farkı ile kıyaslama yapılmıştır
--Avg_Last_3_Discount-Avg_Last_3_Discount 0>Positive, 0<Negative, Neutral
```

In [1]:
USE SampleRetail;
GO

Commands completed successfully.

Total execution time: 00:00:00

In [2]:
SELECT  TOP 0.2 PERCENT *
FROM    sale.order_item
--examine result:
--1 Order, 5 item, 8 quantity, 4 discount category
GO

(10 rows affected)

Total execution time: 00:00:00.020

order_id,item_id,product_id,quantity,list_price,discount
1,1,20,1,249.99,0.20
1,2,8,2,99.99,0.07
1,3,10,2,174.99,0.05
1,4,16,2,599.00,0.05
1,5,4,1,151.99,0.20
2,1,20,1,249.99,0.07
2,2,16,2,599.00,0.05
3,1,3,1,599.00,0.05
3,2,20,1,249.99,0.05
4,1,2,2,136.99,0.10


In [3]:
-- Unpivot the table.  
SELECT Count_of_Order_Item, Datas 
FROM	(
    --Count Orders Item Table 
	select	COUNT(*) Order_Item_Rows
			,COUNT(distinct order_id) Different_Orders
			,COUNT(distinct product_id) Different_Products
			,SUM(quantity) Total_Quantities
			,COUNT(distinct discount) Discount_Category
	from sale.order_item
) like_pvt  
UNPIVOT	(
	Datas 
	FOR Count_of_Order_Item IN (
		Order_Item_Rows
        ,Different_Orders
        ,Different_Products
        ,Total_Quantities 
        ,Discount_Category 
		
	)  
)AS unpvt;    
--examine result: Products, Orders, Discount, Quantities
GO 

(5 rows affected)

Total execution time: 00:00:00.010

Count_of_Order_Item,Datas
Order_Item_Rows,4722
Different_Orders,1615
Different_Products,307
Total_Quantities,7078
Discount_Category,4


In [4]:
drop table if exists #temp_tbl_1;

--Create temp table 1
--WINDOW FUN ile count_order ve total_quantity
select	distinct product_id, discount
        ,count(order_id) OVER(partition by product_id, discount) count_order
        ,sum(quantity) OVER(partition by product_id, discount) total_quantity
INTO	#temp_tbl_1
from sale.order_item

select	TOP 10 PERCENT *
from    #temp_tbl_1
--examine result: count_order, total_quantity, choosed-total_quantity
GO

(849 rows affected)

(85 rows affected)

Total execution time: 00:00:00.037

product_id,discount,count_order,total_quantity
2,0.05,18,26
2,0.07,20,30
2,0.10,18,30
2,0.20,21,32
3,0.05,27,37
3,0.07,19,28
3,0.10,16,23
3,0.20,24,38
4,0.05,22,35
4,0.07,26,37


In [5]:
drop table if exists #temp_tbl_2;

--Create temp table 2
SELECT  product_id, discount, total_quantity
        ,AVG(total_quantity) OVER(PARTITION BY product_id ORDER BY discount ROWS BETWEEN 1 FOLLOWING AND UNBOUNDED FOLLOWING) Avg_Last_3_Discount
        ,DENSE_RANK() OVER(PARTITION BY product_id ORDER BY discount ) [DENSE_RANK]
INTO    #temp_tbl_2
FROM    #temp_tbl_1

SELECT  TOP 10 PERCENT *
FROM    #temp_tbl_2
--examine result: 
--product_id: 2,	discount: 0.05, 	quantity: 26	Avg_Last_3: 30  	[DENSE_RANK]:1 
GO

(849 rows affected)

(85 rows affected)

Total execution time: 00:00:00.024

product_id,discount,total_quantity,Avg_Last_3_Discount,DENSE_RANK
2,0.05,26,30,1
2,0.07,30,31,2
2,0.10,30,32,3
2,0.20,32,NULL,4
3,0.05,37,29,1
3,0.07,28,30,2
3,0.10,23,38,3
3,0.20,38,NULL,4
4,0.05,35,36,1
4,0.07,37,35,2


In [6]:
drop table if exists #temp_tbl_3;

--Create temp table 3
SELECT  product_id, discount, total_quantity
        ,CASE 
                WHEN (Avg_Last_3_Discount - total_quantity) > 0 THEN  'Positive'
                WHEN (Avg_Last_3_Discount - total_quantity) < 0 THEN  'Negative'
                ELSE 'Neutral'
        END [Discount_Effect]
INTO    #temp_tbl_3
FROM    #temp_tbl_2
WHERE   [DENSE_RANK]=1

SELECT  *
FROM    #temp_tbl_3
--examine result: Change Values ('Positive', 'Negative', 'Neutral')
GO

(307 rows affected)

(307 rows affected)

Total execution time: 00:00:00.014

product_id,discount,total_quantity,Discount_Effect
2,0.05,26,Positive
3,0.05,37,Negative
4,0.05,35,Positive
5,0.05,34,Negative
6,0.05,46,Negative
7,0.05,44,Negative
8,0.05,41,Negative
9,0.05,37,Negative
10,0.05,40,Negative
11,0.05,27,Positive


In [7]:
drop table if exists #temp_tbl_4;

--Create temp table 4
SELECT  Discount_Effect
        ,COUNT(product_id) Total_Discount_Effect
INTO    #temp_tbl_4
FROM    #temp_tbl_3
GROUP BY Discount_Effect

SELECT  *
FROM    #temp_tbl_4
ORDER BY 1 DESC
--examine result: Total Effect
GO

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.010

Discount_Effect,Total_Discount_Effect
Positive,92
Neutral,105
Negative,110


In [8]:
select  (
        SELECT  SUM(Total_Discount_Effect)
        FROM    #temp_tbl_4
        WHERE   Discount_Effect NOT IN ('Negative')
        ) 'number of positively affected products'
        ,(
        SELECT  SUM(Total_Discount_Effect)
        FROM    #temp_tbl_4
        WHERE   Discount_Effect IN ('Negative')
        ) 'number of negatively affected products'
GO

(1 row affected)

Total execution time: 00:00:00.008

number of positively affected products,number of negatively affected products
197,110


In [9]:
--result:
--Mostly discount effect look like positively. 
--Discounts have been increasing orders and sales.

Commands completed successfully.

Total execution time: 00:00:00